In [117]:
include("Create_Sample.jl")
include("MIP_SPLIT.jl")
include("Split.jl")
include("GA.jl")
include("Initial.jl")
include("Mutation.jl")
include("Crossover.jl")
include("Neighborhood.jl")
include("Neighborhood_intra.jl")
include("costs.jl")


Calculate_new_cost_3_permute (generic function with 1 method)

In [118]:
instances = [:rat99]
Ms = [2,3,5,7]

for instance in instances
    best_ = 0.0
    Avg_ = 0.0
    worst_ = 0.0
    time_ = 0.0
    for K in Ms
        T = Read_TSPLIB_instance(instance, 1)
        n = size(T)[1]-2
        demands = ones(Int, n)
        W = 150
        h = 0.1
        popsize = (10,50)
        k_tournament = 3
        num_iter = 2000
        Mutation_Chance = 0.1
        num_runs = 10
        avg = 0.0
        best = Inf
        worst = 0.0
        t1 = time() 
        P = Chromosome[]
        for i=1:num_runs
            P = Perform_Genetic_Algorithm(T, demands,K, W, h, popsize, k_tournament, num_iter, Mutation_Chance);
            avg += P[1].fitness
            if P[1].fitness < best
                best = P[1].fitness
            end
            if P[1].fitness > worst
                worst = P[1].fitness
            end
        end
        t2 = time()
        println("Results for ", instance, " ,m=", K)
        println("Best: ", round(best, digits = 1), "  Average: ", round(avg/num_runs, digits = 1), 
            "  Worst: ", round(worst, digits = 1), " , run time= ", round((t2-t1)/num_runs, digits=0))
        best_ += best
        Avg_ += avg/num_runs
        worst_ += worst
        time_ += (t2-t1)/num_runs
    end
    println("Results on all k: Best: ", round(best_/length(Ms), digits = 1), "  Average: ", round(Avg_/length(Ms), digits = 1), 
            "  Worst: ", round(worst_/length(Ms), digits = 1), " , run time= ", round(time_/length(Ms), digits=0))
end

Results for rat99 ,m=2
Best: 740.9  Average: 760.2  Worst: 790.9 , run time= 6.0
Results for rat99 ,m=3
Best: 609.4  Average: 617.0  Worst: 620.5 , run time= 3.0
Results for rat99 ,m=5
Best: 484.9  Average: 490.1  Worst: 500.3 , run time= 4.0
Results for rat99 ,m=7
Best: 453.7  Average: 455.5  Worst: 462.9 , run time= 4.0
Results on all k: Best: 572.2  Average: 580.7  Worst: 593.7 , run time= 4.0


In [119]:
for tour in P[1].tours
    t1 = copy(tour.Sequence);
    pushfirst!(t1, 0)
    push!(t1, n+1)

    z1 = 0.0
    for i=1:length(t1)-1
        z1 += T[t1[i]+1, t1[i+1]+1]
    end

    println(z1, " ,  ", tour.cost)
end

LoadError: BoundsError: attempt to access 0-element Vector{Chromosome} at index [1]

In [83]:
function Calculate_new_cost_3_permute1(tour::Vector{Int}, cost::Float64, S1::Vector{Int}, S2::Vector{Int}
        ,k1::Int, T::Matrix{Float64}, n_nodes::Int)
    if S1 == S2 
        return cost
    end
    nt = length(tour)
    cost = cost - T[S1[1]+1, S1[2]+1] - T[S1[2]+1, S1[3]+1] + T[S2[1]+1, S2[2]+1] + T[S2[2]+1, S2[3]+1]
    
    if k1 == 1
        cost = cost - T[1, S1[1]+1] + T[1, S2[1]+1] - T[S1[3]+1, tour[4]+1] + T[S2[3]+1, tour[4]+1] 
    elseif k1 == nt-2
        cost = cost - T[S1[3]+1, n_nodes+2] + T[S2[3]+1, n_nodes+2] - T[tour[nt-3]+1, S1[1]+1] + T[tour[nt-3]+1, S2[1]+1]
    else
        cost = cost - T[tour[k1-1]+1, S1[1]+1] + T[tour[k1-1]+1, S2[1]+1] - T[S1[3]+1, tour[k1+3]+1] + T[S2[3]+1, tour[k1+3]+1] 
    end
    return cost
end



function Calculate_new_cost_3_permute2(tour::Vector{Int}, cost::Float64, S1::Vector{Int}, S2::Vector{Int}
        ,k1::Int, T::Matrix{Float64}, n_nodes::Int)

    nt = length(tour)
    
    t1 = copy(tour)
    t1[k1:k1+2] = S2
    pushfirst!(t1, 0)
    push!(t1, n_nodes+1)
    z1 = 0.0
    for i=1:length(t1)-1
        z1 += T[t1[i]+1, t1[i+1]+1]
    end
    
    return z1
end

Calculate_new_cost_3_permute2 (generic function with 1 method)

In [38]:
instance = :rat99
T = Read_TSPLIB_instance(instance, 1)
n_nodes = size(T)[1]-2
tsp_tour = find_tsp_tour1(T)
demands = ones(Int, n_nodes)
W = 150
K = 2
mu = 10
P, _ = Generate_initial_population(T, demands, K, W, mu, tsp_tour);

In [39]:
P[1].tours

2-element Vector{Tour}:
 Tour([9, 10, 11, 12, 21, 20, 19, 18, 27, 36  …  82, 90, 91, 92, 93, 94, 95, 96, 97, 98], 823.6584356071808)
 Tour([89, 88, 78, 87, 86, 85, 84, 83, 75, 76  …  17, 8, 7, 6, 5, 4, 13, 3, 2, 1], 806.3918274353607)

In [74]:
tour1 = copy(P[1].tours[1].Sequence)
tour2 = copy(P[1].tours[1].Sequence)
cost1 = P[1].tours[1].cost
cost2 = P[1].tours[1].cost



823.6584356071808

In [85]:
include("Neighborhood.jl")

tt1 = 0.0
tt2 = 0.0
t1 = time()
for i=1:10000000
    k1 = rand(1:length(tour1)-2)
    S1 = copy(tour1[k1:k1+2])
    S2 = shuffle(S1)

    new_cost1 = Calculate_new_cost_3_permute1(tour1, cost1, S1, S2, k1, T, n_nodes)
#     z1 = Calculate_new_cost_3_permute2(tour1, cost1, S1, S2, k1, T, n_nodes)

#     if round(new_cost1, digits = 6) != round(z1, digits = 6)
#         println(k1)
#         println(round(new_cost1, digits = 6), "   ", round(z1, digits = 6))
#     end
end

println("time for 1:", time()- t1)

t2 = time()
for i=1:10000000
    k1 = rand(1:length(tour1)-2)
    S1 = copy(tour1[k1:k1+2])
    S2 = shuffle(S1)

    z1 = Calculate_new_cost_3_permute2(tour1, cost1, S1, S2, k1, T, n_nodes)

end

println("time for 2:", time()- t2)


time for 1:4.017000198364258
time for 2:9.663999795913696


In [42]:
length(tour1)

49

In [29]:
# a = [1,2,3,4]

p = [100, 5, 10, 1, 1000]

for i=1:100
    w = weights(p)
    r = sample(1:5, w)
    println(r)
end

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
1
5
5
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
1
5
5
3
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [19]:
sample(a, p)

LoadError: MethodError: no method matching sample(::Vector{Int64}, ::Vector{Int64})
[0mClosest candidates are:
[0m  sample(::AbstractArray) at C:\Users\sasanm\.julia\packages\StatsBase\XgjIN\src\sampling.jl:432
[0m  sample(::AbstractArray, [91m::Integer[39m; replace, ordered) at C:\Users\sasanm\.julia\packages\StatsBase\XgjIN\src\sampling.jl:513
[0m  sample(::AbstractArray, [91m::Tuple{Vararg{Int64, N}} where N[39m; replace, ordered) at C:\Users\sasanm\.julia\packages\StatsBase\XgjIN\src\sampling.jl:534
[0m  ...

In [24]:
w = weights(p)
sample(a, w)

1

In [25]:
w

4-element Weights{Int64, Int64, Vector{Int64}}:
 100
   5
  10
   1

In [90]:
min(1,4)

1